In [9]:
from rdkit import Chem
import numpy as np
import pandas as pd
from calculate_delta_energy.gap_deltaenergy import *
import pickle

In [10]:
# 在这里输入你想要研究的分子的SMILES表示
# 提取特征
# 预测分子global毒性 预测值越接近0意味着无毒，越接近1意味着有毒
your_smis = ['[H][C@@]12CCC[C@]1([H])N([C@@H](C2)C(O)=O)C(=O)[C@H](C)N[C@@H](CCC1=CC=CC=C1)C(=O)OCC']
your_mols = [Chem.MolFromSmiles(smi) for smi in your_smis]
print(Chem.MolToSmiles(your_mols[0]))
your_nbegapenergy, your_smiles_list =GetNBEandGapEnergy_asfeatures(your_mols, your_smis)

df_your_smi = pd.DataFrame({'standard_smiles':your_smiles_list})
df_your_gapenergy = pd.DataFrame(your_nbegapenergy, columns= col_list)
df_your_gapenergy = pd.concat([df_your_smi, df_your_gapenergy],axis=1)

df_your_gapenergy_fe = FE(df_your_gapenergy)
print(df_your_gapenergy_fe.shape)
df_your_all = descri(df_your_gapenergy_fe)



CCOC(=O)[C@H](CCc1ccccc1)N[C@@H](C)C(=O)N1[C@H](C(=O)O)C[C@@H]2CCC[C@@H]21
(1, 226)


In [11]:
print(f"分子gap-delta-energy特征")
df_your_gapenergy_fe

分子gap-delta-energy特征


,standard_smiles,sx_ave1,sx_ave2,sx_ave3,sx_ave4,sx_ave5,sx_ave6,sx_ave7,sx_ave8,sx_ave9,...,nx_std6*nx_std7,nx_std6*nx_std8,nx_std6*nx_std9,nx_std6*nx_std10,nx_std7*nx_std8,nx_std7*nx_std9,nx_std7*nx_std10,nx_std8*nx_std9,nx_std8*nx_std10,nx_std9*nx_std10
0,[H][C@@]12CCC[C@]1([H])N([C@@H](C2)C(O)=O)C(=O...,81.378151,74.631868,99.044554,99.112821,93.343434,81.126263,85.461957,93.365714,87.024691,...,12227.933192,11498.54697,11001.515892,11977.062418,12555.846723,12013.113276,13078.362015,11296.540886,12298.248408,11766.649791


In [12]:
print(f"分子的总特征")
df_your_all

分子的总特征


,standard_smiles,sx_ave1,sx_ave2,sx_ave3,sx_ave4,sx_ave5,sx_ave6,sx_ave7,sx_ave8,sx_ave9,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,[H][C@@]12CCC[C@]1([H])N([C@@H](C2)C(O)=O)C(=O...,81.378151,74.631868,99.044554,99.112821,93.343434,81.126263,85.461957,93.365714,87.024691,...,0,0,0,0,0,0,0,0,0,0


In [13]:
xgb_gapen_important_features = np.load("data/xgb_gapen_important_features.npy", allow_pickle=True)
xgb_descri_important_features = np.load("data/xgb_descri_important_features.npy", allow_pickle=True)

feature_names = np.load("./data/feature_names.npy", allow_pickle=True)
df_mean= np.load("./data/feature_df_mean.npy")
df_std= np.load("./data/feature_df_std.npy")

def get_feature(df_your_all = df_your_all):
    features = df_your_all.iloc[:,1:]
    features = features[feature_names]
    
    # features = np.array(features)
    print(features.shape)
    # df_mean = 
    # df_std = 
    features = (features - df_mean) / df_std
    features = features.fillna(0.0)
    return features

your_features = get_feature()

(1, 429)


In [14]:
print("经过特征选择之后的特征")
your_features

经过特征选择之后的特征


,sx_ave1,sx_ave2,sx_ave3,sx_ave4,sx_ave5,sx_ave6,sx_ave7,sx_ave8,sx_ave9,sx_ave10,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,-0.402037,-0.138743,0.218359,0.3602,0.172622,-0.081467,0.141281,0.478555,0.324153,0.096932,...,-0.191016,-0.068355,-0.048275,-0.054313,-0.011301,-0.101031,-0.028617,-0.067561,-0.407879,-0.074021


In [15]:
# load XGB model with selected features
def pickle_XGBmodel_test(clf_str,  features = your_features):
    with open('models/'+clf_str+'.pkl', 'rb') as f:
        clf = pickle.load(f)
    if clf_str == 'xgb_gap':
        X_test =  features[xgb_gapen_important_features]
    elif clf_str == 'xgb_des':
        X_test =  features[xgb_descri_important_features]
    y_test_pred = clf.predict(X_test)
    y_test_probas = clf.predict_proba(X_test)[:, 1]
    print(f"{clf_str}分类器预测毒性概率: {y_test_probas}")

    return y_test_probas

xgb_gapen_test_proba = pickle_XGBmodel_test('xgb_gap')
xgb_descri_test_proba = pickle_XGBmodel_test('xgb_des')

xgb_gap分类器预测毒性概率: [0.7102969]
xgb_des分类器预测毒性概率: [0.23002003]


In [16]:
# load the LR model
# 最终毒性预测概率
def pickle_LRmodel_test(X_test_proba):

    with open('models/'+'XGB_LR'+'.pkl', 'rb') as f:
        clf = pickle.load(f)
    y_test_pred = clf.predict(X_test_proba)
    y_test_probas = clf.predict_proba(X_test_proba)[:, 1]
    print("XGB-LR-stacking 最终的毒性预测概率" ,y_test_probas)
    return y_test_probas

X_test_proba = np.concatenate([xgb_gapen_test_proba[:,np.newaxis], xgb_descri_test_proba[:,np.newaxis]], axis=1)
y_test_probas = pickle_LRmodel_test(X_test_proba)

XGB-LR-stacking 最终的毒性预测概率 [0.40326961]
